#TAREA 4
Nombre: Nicolás Sumonte

- En primer lugar, importamos PyopenCl y otras librerias necesarias

In [1]:
!pip install pyopencl

In [2]:
import numpy as np
from time import time

- Usamos la función dada en la tarea

In [3]:
def generate_matrix(dim):
  from scipy.stats import ortho_group
  from scipy.sparse import spdiags
  a = ortho_group.rvs(dim, random_state=0)
  b = np.linspace(1., 10., dim)
  return a @ spdiags(b, 0, dim, dim) @ a.T

- definimos el metodo de la potencia, utilizando delta la medida de error, iteracion a iteración para terminar el algoritmo

In [4]:
def power_method_numpy(A, b0, delta):
  mu = 0
  while True:
    Ab = np.dot(A,b0)
    btb = np.dot(np.transpose(b0),b0)
    btAb = np.dot(np.transpose(b0),Ab)
    mu_new = btAb / btb
    norm = np.linalg.norm(Ab)
    b0 = Ab / norm
    if np.abs(mu - mu_new) < delta:
      break
    else:
      mu = mu_new
  return mu

In [5]:
A = generate_matrix(1000)
b = np.ones(1000)
delta = 0.001

- Vemos el resultado del metodo de la potencia

In [6]:
power_method_numpy(A,b,delta)

9.938052941072023

- Implementamos el método con Pyopencl

In [24]:
import pyopencl as cl
import pyopencl.array as cl_array
platform = cl.get_platforms()[0]
device = platform.get_devices(cl.device_type.GPU)[0]
print("Platform name:", platform.name)
print("Device name:", device.name)
print("Maximum work group size:", device.max_work_group_size)
platforms = cl.get_platforms()
ctx = cl.Context(dev_type=cl.device_type.GPU, properties=[(cl.context_properties.PLATFORM, platforms[0])])

queue = cl.CommandQueue(ctx)
workgroup_size = 30
n_workgroups = 8
matrix_size = 1000

A = generate_matrix(matrix_size)
b = np.ones(matrix_size)
 ### función que realiza el padding de los datos
def padding(A, b, workgroup_size, matrix_size):
  matrix_size_new = matrix_size
  if matrix_size_new % workgroup_size == 0:
    return A,b,matrix_size_new
  else:
    while matrix_size_new % workgroup_size != 0:
      b = np.append(b,0)
      A = np.c_[A, np.zeros(matrix_size)] 
      matrix_size_new +=1
    return A, b, matrix_size_new

print("\nWorkgroup size:", workgroup_size)
print("Matrix size:", matrix_size)
kernel = """
__kernel void matvec(__global double *matrix,
                     __global double *vector,
                     __global double *output
                     )
{
  int dim = get_global_size(0);
  int global_id = get_global_id(0);
  double local_sum = 0.0;
  int offset = global_id * dim;
  for(int i = 0; i < dim; i++){
    local_sum += matrix[i + offset] * vector[i];
  }
  output[global_id] = local_sum;
}
__kernel void norm_calc(__global double *res, __global double *res1)
{
  int dim = get_global_size(0);
  int global_id = get_global_id(0);
  double sqrt_sum = 0.0;
  for (int i = 0; i < dim; i++)
  {
    sqrt_sum += res[i]*res[i];
  }
  barrier(CLK_LOCAL_MEM_FENCE);
  res1[0] = sqrt(sqrt_sum);
}
"""

A, b, matrix_size = padding(A,b, workgroup_size, matrix_size)
result = np.zeros(matrix_size)
result_1 = np.zeros(1)
prg = cl.Program(ctx, kernel).build()
cl_matrix = cl_array.to_device(queue, A.ravel())
cl_vector = cl_array.to_device(queue, b)
cl_AB = cl_array.empty(queue, matrix_size, dtype=np.float64)
cl_maxvec = cl_array.empty(queue, 1, dtype = np.float64)

def opencl_grad(itr,A,b,matrix_size):
  counter = 0
  times_1 = []
  times_2 = []
  while counter < itr:
    t0 = time()
    event = prg.matvec(queue,
                       (matrix_size,), (workgroup_size,),
                       cl_matrix.data, cl_vector.data, cl_AB.data)
    t1 = time()
    times_1.append(np.abs(t0 - t1))
    events = [event]
    cl.enqueue_copy(queue, result, cl_AB.data, wait_for=events);
    Ab = result
    btb = np.dot(np.transpose(b),b)
    btAb = np.dot(np.transpose(b),Ab)
    t2 = time()
    event = prg.norm_calc(queue,
                          (matrix_size,), (workgroup_size,),
                          cl_vector.data, cl_maxvec.data)
    t4 = time()
    times_2.append(np.abs(t2 - t4))
    events = [event]
    result_1 = cl_maxvec.get()[0]
    b = Ab / result_1
    mu = btAb / btb
    cl.enqueue_copy(queue, cl_vector.data, b, wait_for=events);
    counter += 1
  time1 = sum(times_1) / len(times_1)
  time2 = sum(times_2) / len(times_2)
  return mu, time1, time2

Platform name: NVIDIA CUDA
Device name: Tesla K80
Maximum work group size: 1024

Workgroup size: 30
Matrix size: 1000


- Aplicamos el metodo con 1500 iteraciones e imprimimos el error del método y los tiempos promedios dentro de los kernels

In [25]:
mu_by_pyopencl,time1,time2 = opencl_grad(1500, A, b, matrix_size)
error = 10 - mu_by_pyopencl
print(f'el error de pyopencl es: {error}')
print(f'el tiempo promedio dentro del kernel multiplicación es: {time1}')
print(f'el tiempo promedio dentro del kernel de norma es : {time2}')


1020
el error de pyopencl es: 0.008050345991511776
el tiempo promedio dentro del kernel multiplicación es: 0.00011555226643880209
el tiempo promedio dentro del kernel de norma es : 0.00011650196711222331


- Definimos la función analizar metodos que nos permite calcular los tiempos de distintos metodos

In [31]:
def analizar_metodos(dim_matrix, workgroupsize,itr):

  matrix_size_1 = dim_matrix
  A = generate_matrix(matrix_size_1)
  b = np.ones(matrix_size_1)
  t0 = time()
  power_method_numpy(A,b,delta)
  t1 = time()
  tiempo_power = t1 - t0
  print(f'power method tarda: {tiempo_power}')

  platform = cl.get_platforms()[0]
  device = platform.get_devices(cl.device_type.GPU)[0]
  platforms = cl.get_platforms()
  ctx = cl.Context(dev_type=cl.device_type.GPU, properties=[(cl.context_properties.PLATFORM, platforms[0])])
  queue = cl.CommandQueue(ctx)

  workgroup_size = workgroupsize
  n_workgroups = 8

  A, b, matrix_size = padding(A, b, workgroup_size, matrix_size_1)
  result = np.zeros(matrix_size)
  result_1 = np.zeros(1)
  prg = cl.Program(ctx, kernel).build()
  cl_matrix = cl_array.to_device(queue, A.ravel())
  cl_vector = cl_array.to_device(queue, b)
  cl_AB = cl_array.empty(queue, matrix_size, dtype=np.float64)
  cl_maxvec = cl_array.empty(queue, 1, dtype = np.float64)
  t2 = time()
  opencl_grad(itr, A, b, matrix_size)
  t3 = time()
  tiempo_pyopen = t3 - t2
  print(f'pyopencl method tarda: {tiempo_pyopen}')
  return tiempo_power,tiempo_pyopen

In [32]:
tiempos_pyopen = []
tiempos_power = []
lista_dim_matrix = [1000]

for k in lista_dim_matrix:
  t_py, t_po = analizar_metodos(k, 30, 1000)
  tiempos_power.append(t_po)
  tiempos_pyopen.append(t_py)

power method tarda: 0.020765066146850586
1020
pyopencl method tarda: 1.028076410293579
